# channelmapper: A workflow for mapping channels in satellite imagery

Zoltan Sylvester, Quantitative Clastics Laboratory, UT Austin

January 2021

## Requirements

* opencv
* rivamap
* numpy
* matplotlib
* jupyter
* geopandas
* scipy
* scikit-image
* shapely
* descartes
* sklearn
* librosa (https://github.com/librosa/librosa; install with: conda install -c conda-forge librosa)

In [1]:
import cv2
from rivamap import preprocess, singularity_index, delineate, georef, visualization
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage.measurements
from scipy.signal import savgol_filter
import scipy.interpolate
from skimage import morphology
from scipy.spatial import distance

import channelmapper as cm

%matplotlib qt

## 1. Find centerline

### 1.1. Read images, compute water index, compute singularity index, create and save centerline map

The Landsat scene used as an example below can be downloaded from here:

https://www.dropbox.com/sh/e0sv7zx44v4jb6r/AAAoI2VyiE6ZfVqVDehVz_kra?dl=0

The edited/QC-d 'centerlines.TIF' and 'water.TIF' files are available here:

https://www.dropbox.com/s/tefn16ypnpfzxeg/centerlines.TIF?dl=0

https://www.dropbox.com/s/ut4runvom5056ba/water.TIF?dl=0

In [2]:
dirname = '/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/'
tilename = 'LC08_L1TP_232070_20180622_20180623_01_RT'

# bands 3 and 6 for Landsat 8; bands 2 and 5 for Landsat 4-5
# for Landsat 8:
B2 = cv2.imread(dirname+tilename+'/'+tilename+'_B3.TIF', cv2.IMREAD_UNCHANGED) 
B5 = cv2.imread(dirname+tilename+'/'+tilename+'_B6.TIF', cv2.IMREAD_UNCHANGED) 
# for Landsat 4-5:
# B2 = cv2.imread(dirname+tilename+'/'+tilename+'_B2.TIF', cv2.IMREAD_UNCHANGED) 
# B5 = cv2.imread(dirname+tilename+'/'+tilename+'_B5.TIF', cv2.IMREAD_UNCHANGED) 

#Compute the modified normalized difference water index of the input and save the result if needed.
I1 = preprocess.mndwi(B2, B5)
gm = georef.loadGeoMetadata(dirname+tilename+'/'+tilename+'_B4.TIF')
# georef.saveAsGeoTiff(gm, I1.astype('float'), dirname+tilename+'/'+tilename[17:25]+"_mndwi.TIF")

# Create the filters that are needed to compute the multiscale singularity index and apply the index to 
# extract curvilinear structures from the input image. The singularity index function returns the overall 
# singularity index response, width estimates, and channel orientation for each pixel whether or not they 
# are river centerlines. We will find the river centerlines in the next step. You can save or view the 
# overall singularity index response if needed:
filters = singularity_index.SingularityIndexFilters()
psi, widthMap, orient = singularity_index.applyMMSI(I1, filters)

# Extract and threshold centerlines to delineate rivers:
nms = delineate.extractCenterlines(orient, psi)
centerlines = delineate.thresholdCenterlines(nms)

# label objects in image:
s = [[1,1,1],[1,1,1],[1,1,1]]
labeled_array, num_features = scipy.ndimage.measurements.label(centerlines.astype('int'), structure=s)
labels = np.unique(labeled_array)
# measure how big the objects are:
sizes = np.bincount(labeled_array.flat)
# get rid of the very small pieces:
t = labeled_array.copy()
for i in range(1,len(sizes)):
    if sizes[i]>=50:
        t[t==i] = -1
t[t>0]=0
t[t==-1]=1
# display resulting image:
plt.figure(figsize=(15,15))
plt.imshow(t, cmap='gray')

Processing scale: 0
Processing scale: 1
Processing scale: 2
Processing scale: 3
Processing scale: 4
Processing scale: 5
Processing scale: 6
Processing scale: 7
Processing scale: 8
Processing scale: 9
Processing scale: 10
Processing scale: 11
Processing scale: 12
Processing scale: 13
Processing scale: 14


In [4]:
# save image as a TIF file:
georef.saveAsGeoTiff(gm, t.astype('float'), "centerlines.TIF")

### 1.2. QC the centerline in Photoshop

Open the centerline file in Photoshop and get rid of any gaps and spots where 4 neighboring pixels are all part of the centerline. The goal is to have one continuous line of pixels, with no bifurcations or thicker segments, from the beginning of the main channel we are mapping to the end. Use the 'pencil' tool inPhotohsop, with the line width set to 1 pixel and zoom in so that you can see the pixels.

### 1.3. Track centerline (after the Photoshop QC)

In [5]:
# label centerline image to find main channel centerline
# the goal of this cell is only to figure out whether further Photoshop edits are needed to 
# get the whole centerline before we proceed to tracking
s = [[1,1,1],[1,1,1],[1,1,1]]
sk = cv2.imread("centerlines.TIF", cv2.IMREAD_UNCHANGED)
sk[(sk!=0) & (sk!=1)] = 0 # make sure that sk only has zeros and ones
sk_labeled, nl = scipy.ndimage.measurements.label(sk, structure=s)
sk_sizes = np.bincount(sk_labeled.flat)
t = sk_labeled.copy()
plt.figure(figsize=(15,15))
plt.imshow(t, cmap='viridis',vmin=0,vmax=200)

In [6]:
# find largest object (we assume that this is the channel we are looking for, but in rare cases that is not true)
ind = np.where(sk_sizes[1:]==np.max(sk_sizes[1:]))[0][0] + 1
len(t[t==ind]) # number of pixels in largets centerline in image

13677

In [8]:
# once we made sure that the centerline image contains the correct & entire centerline, we proceed to detect it
# re-label image
s = [[1,1,1],[1,1,1],[1,1,1]]
sk = cv2.imread("centerlines.TIF", cv2.IMREAD_UNCHANGED)
sk[(sk!=0) & (sk!=1)] = 0 # make sure that sk only has zeros and ones
sk_labeled, nl = scipy.ndimage.measurements.label(sk, structure=s)
sk_sizes = np.bincount(sk_labeled.flat)
t = sk_labeled.copy()
t[t==ind] = -1  # change the 'ind' value here to get the correct centerline
t[t>0]=0 # set 'background' to zero
t[t==-1]=1 # set centerline to '1'

y_pix,x_pix = np.where(t==1) # 

# find starting pixel at bottom of image (if it is the bottom of the image)
ind = np.where(y_pix==np.max(y_pix))[0][0]
x0 = x_pix[ind]
y0 = y_pix[ind]
print(x0, y0)

# find starting pixel on left side of image
# ind = np.where(x_pix==np.min(x_pix))[0][0]
# x0 = x_pix[ind]
# y0 = y_pix[ind]
# print x0, y0

# sometimes it is best to locate the starting pixel of the centerline and enter the values manually:
# x0 = 2500 # x coordinate of starting point for centerline
# y0 = 5999
# y_pix,x_pix = np.where(t==1)

# 'track' the centerline (= order the pixels from beginning to end)
start_ind = np.where((x_pix==x0) & (y_pix==y0))[0][0] # index of starting point
# distance matrix for all points on centerline:
dist = distance.cdist(np.array([x_pix,y_pix]).T,np.array([x_pix,y_pix]).T)
# every point is closest to itself, so we want to set the distances along the diagonal of the matrix to 
# a number significantly larger than zero:
dist[np.diag_indices_from(dist)]=100.0  
ind = start_ind # start with the first point
clinds = [ind] # list that we use to collect all the centerline indices 
count = 0
while count<len(x_pix): # do this while we find all the points in x_pix and y_pix
    d = dist[ind,:].copy() # all distances from point of interest (with index 'ind')
    # we want to prevent the tracking to go backwards, so the distances to the last two 
    # points are increased with 100:
    if len(clinds)>2: 
        d[clinds[-2]]=d[clinds[-2]]+100.0
        d[clinds[-3]]=d[clinds[-3]]+100.0
    if len(clinds)==2:
        d[clinds[-2]]=d[clinds[-2]]+100.0
    ind = np.argmin(d) # find index of point that is closest to the point of interest
    clinds.append(ind) # add the new point to the list
    count=count+1
x_pix = x_pix[clinds]
y_pix = y_pix[clinds]

plt.figure(figsize=(15,15))
plt.imshow(sk,cmap='viridis')
plt.plot(x_pix,y_pix,'r');

2671 6729


### 1.4. Get image corners in UTM coordinates

In [9]:
# left x coordinate, delta x, upper y coordinate, delta y; 174900.000, -1655400.000 are the "official" coordinates so 
# 'gm.geotransform' must be the coordinates of the edges of the pixels, whereas the 'official' coordinates refer to 
# centers of the pixels
# dirname = '/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/'
# tilename = 'LC08_L1TP_232070_20180622_20180623_01_RT'
gm = georef.loadGeoMetadata(dirname+tilename+'/'+tilename+'_B4.TIF')

left_utm_x = gm.geotransform[0]
upper_utm_y = gm.geotransform[3]
delta_x = gm.geotransform[1]
delta_y = gm.geotransform[5]
nx = I1.shape[1]
ny = I1.shape[0]
right_utm_x = left_utm_x + delta_x*nx
lower_utm_y = upper_utm_y + delta_y*ny

## 2. Find banks

### 2.1. Create binary water index image

In [10]:
water = I1.copy()
water[water>0]=1
water[water<=0]=0

### Dilate centerline image
cl_dilated = morphology.binary_dilation(t, morphology.square(40)).astype(np.uint8)

### Set water index image to zero in areas farther away from centerline
water[np.where(cl_dilated==0)]=0

### Find main connected component in water image and delete the rest
s = [[1,1,1],[1,1,1],[1,1,1]]
water_labeled, nl = scipy.ndimage.measurements.label(water, structure=s)
water_sizes = np.bincount(water_labeled.flat)
ind = np.where(water_sizes[1:]==np.max(water_sizes[1:]))[0][0] + 1
water=np.zeros(np.shape(water))
water[np.where(water_labeled==ind)]=1

plt.figure(figsize=(15,15))
plt.imshow(water,cmap='gray')

### 2.2. Save water index image to file (for QC in Photoshop)

In [48]:
georef.saveAsGeoTiff(gm, water.astype('float'), "water.TIF")

### 2.3. QC water image in Photoshop

Open the water image in Photoshop and (1) delete any bifurcations or extra bits that would make it difficult to get a nice channel bank; (2) delete all islands in the channel. The goal is to have one white band across the image, with relatively smooth boundaries and no islands. Use the 'pencil' tool inPhotohsop, with the line width set to a small number of pixels and zoom in so that you can see the pixels. You can use the keyboard shortcut 'x' to switch back and forth between the black and white pencil.

### 2.4. Read back QC-d water index image and find edges

In [11]:
filename = "water.TIF"
water = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
plt.figure(figsize=(15,15))
plt.grid('off')
plt.imshow(water,cmap='gray')
C = plt.contour(water, [0.5], linewidths=1, colors='r')

### 2.5. Find and separate left bank and right bank

In [12]:
bank_x = C.allsegs[0][0][:,0] # generate x coordinates from contour
bank_y = C.allsegs[0][0][:,1] # generate y coordinates from contour

plt.figure(figsize=(15,15))
plt.plot(bank_x,bank_y,'.-')
plt.axis('equal')
plt.gca().invert_yaxis()

plt.plot(bank_x[0],bank_y[0],'or') # plot starting point of x and y coordinates
# plt.plot(x_pix[0],y_pix[0],'ok')
# plt.plot(x_pix[-1],y_pix[-1],'ob')

In [13]:
inds=[] # initialize list of 'corner' points

This next cell has to be executed four times, to find the four cornerpoints of the channel boundary.

Start first with the **upstream** end of the channel, and the point on the **left** bank of the channel.
Continue in an **anticlockwise** direction, until you picked all the four points.

To start, you need to zoom in to the upstream end of the channel so that you can see the points clearly. After picking the first two points (for the upstream end), you need to zoom into the downstream end.

In [17]:
ind1 = plt.ginput(1)
# bank_x and bank_y either fall on the edge or the middle of a pixel, so we can
# round the coordinates in 'ind1' and divide by 2 to find the closest point in the channel outline:
ind1 = np.round(np.array(ind1)*2)/2.0
ind1 = np.where((bank_x == ind1[0][0]) & (bank_y == ind1[0][1]))[0][0]
inds.append(ind1)

In [19]:
# check that 'inds' has four indices and they make sense
inds

[15088, 15074, 0, 30254]

In the next cells, we split the channel object outline into left bank and right bank.
Depending on how the coordinates in the contourline were ordered, you need to use one of the following three cells. Usually the first one works.

In [20]:
# this is the cell that works with the example dataset
rb_x = bank_x[inds[1]:inds[2]:-1] 
lb_x = bank_x[inds[0]:inds[3]]
rb_y = bank_y[inds[1]:inds[2]:-1]
lb_y = bank_y[inds[0]:inds[3]]

plt.figure()
plt.plot(rb_x,rb_y)
plt.plot(lb_x,lb_y)
plt.plot(rb_x[0],rb_y[0],'ro')
plt.axis('equal')
plt.gca().invert_yaxis()

In [499]:
rb_x = np.hstack((bank_x[0:inds[1]][::-1], bank_x[inds[2]:-1][::-1]))
lb_x = bank_x[inds[0]:inds[3]]
rb_y = np.hstack((bank_y[0:inds[1]][::-1], bank_y[inds[2]:-1][::-1] ))
lb_y = bank_y[inds[0]:inds[3]]

plt.figure()
plt.plot(rb_x,rb_y)
plt.plot(lb_x,lb_y)
plt.plot(rb_x[0],rb_y[0],'ro')
plt.axis('equal')
plt.gca().invert_yaxis()

In [103]:
rb_x = bank_x[inds[1]:inds[2]:-1] 
lb_x = np.hstack((bank_x[inds[0]:-1], bank_x[0:inds[3]] ))
rb_y = bank_y[inds[1]:inds[2]:-1]
lb_y = np.hstack((bank_y[inds[0]:-1], bank_y[0:inds[3]] ))

plt.figure()
plt.plot(rb_x,rb_y)
plt.plot(lb_x,lb_y)
plt.plot(rb_x[0],rb_y[0],'ro')
plt.axis('equal')
plt.gca().invert_yaxis()

## 3. Resample and smooth banks and centerline

In [21]:
import scipy.interpolate

def resample_and_smooth(x,y,delta_s,smoothing_factor):
    dx = np.diff(x); dy = np.diff(y)      
    ds = np.sqrt(dx**2+dy**2)
    tck, u = scipy.interpolate.splprep([x,y],s=smoothing_factor) # parametric spline representation of curve
    unew = np.linspace(0,1,1+int(sum(ds)/delta_s)) # vector for resampling
    out = scipy.interpolate.splev(unew,tck) # resampling
    xs = out[0]
    ys = out[1]
    return xs, ys

In [22]:
deltas = 25.0 # sampling distance

plt.figure()
plt.imshow(I1, extent=[left_utm_x,right_utm_x,lower_utm_y,upper_utm_y], interpolation='none')
plt.grid('off')

rbx = left_utm_x + 0.5*delta_x + rb_x*delta_x 
rby = upper_utm_y + 0.5*delta_y + rb_y*delta_y 
plt.plot(rbx,rby,'.-g')
rbx = savgol_filter(rbx, 11, 3)
rby = savgol_filter(rby, 11, 3)
rbxs, rbys = resample_and_smooth(rbx,rby,deltas,0.5*1000000)
plt.plot(rbxs,rbys,'.-r')

lbx = left_utm_x + 0.5*delta_x + lb_x*delta_x 
lby = upper_utm_y + 0.5*delta_y + lb_y*delta_y 
plt.plot(lbx,lby,'.-g')
lbx = savgol_filter(lbx, 11, 3)
lby = savgol_filter(lby, 11, 3)
lbxs, lbys = resample_and_smooth(lbx,lby,deltas,0.5*1000000)
plt.plot(lbxs,lbys,'.-r')

x = left_utm_x + 0.5*delta_x + x_pix*delta_x 
y = upper_utm_y + 0.5*delta_y + y_pix*delta_y 
plt.plot(x,y,'.-g')
x = savgol_filter(x, 21, 3)
y = savgol_filter(y, 21, 3)
plt.plot(x,y,'.-y')
xs, ys = resample_and_smooth(x,y,deltas,0.5*1000000)
plt.plot(xs,ys,'.-r')

In [23]:
# replace variables with smoothed versions
x = xs
y = ys
rbx = rbxs
rby = rbys
lbx = lbxs
lby = lbys

## 4. Make sure centerline and banks have roughly the same lengths

This bit is needed because sometimes the centerline is longer than the banks.

In [28]:
# use a KD-tree to find closest point to location of click:
from sklearn.neighbors import KDTree
tree = KDTree(np.vstack((x, y)).T)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(x, y, '.-') 
ax.plot(lbx, lby, 'k')
ax.plot(rbx, rby, 'k')
plt.axis('equal');

Zoom into the area of interest before executing this next cell.

In [29]:
# click on point where you want to cut the centerline
ind = plt.ginput(1)
nearest_dist, nearest_ind = tree.query(np.reshape([ind[0][0],ind[0][1]], (1, -1)))
ind = nearest_ind[0][0]
plt.plot(x[ind], y[ind], 'ro')
print(ind)

13895


In [30]:
# get rid of the extra bit of centerline (at the downstream end):
x = x[:ind+1]
y = y[:ind+1]

In [26]:
# get rid of the extra bit of centerline (at the upstream end):
x = x[ind:]
y = y[ind:]

In [31]:
# plotting for QC:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(x, y, '.-') 
ax.plot(lbx, lby, 'k')
ax.plot(rbx, rby, 'k')
plt.axis('equal');

## 5. Create channel segment polygons for one channel

### 5.1. Estimate left- and right-widths, using dynamic time warping

In [38]:
rbw, lbw, pnr, qnr, pnl, qnl = cm.estimate_half_widths(x, y, rbx, lbx, rby, lby)

### 5.2. Create channel segment polygons

In [39]:
polys = cm.create_channel_segment_polygons(x, y, rbx, rby, lbx, lby, lbw, rbw, deltas=25.0, extra_width=50.0)

### 5.3. Crop polygons to the channel width

In [43]:
# if you get a "TopologyException" error here, it means that the input channel banks have a self-intersection 
# at the location given by the coordinates in the error message. This needs to be eliminated before moving on, 
# e.g., by editing the line manually in QGIS.

ch = cm.create_channel_polygon(lbx, lby, rbx, rby)
cropped_polys = cm.crop_polygons_to_channel_width(polys, ch)

### 5.4. Find overlapping polygons

In [44]:
inds = cm.find_overlapping_polys(cropped_polys, 1.0) 

In [45]:
inds

array([ 530,  531,  532,  533,  534,  535,  536,  538,  539,  540,  541,
        542,  543,  544,  582,  583,  584,  585,  588,  589,  590,  591,
        592,  593, 1644, 1645, 1646, 1647, 1648, 1649, 1650, 1651, 1652,
       1653, 1654, 1657, 1658, 1659, 1660, 1661, 1662, 1663, 1664, 3030,
       3031, 3032, 3033, 3034, 3035, 3036, 3037, 3038, 3039, 3042, 3043,
       3044, 3045, 3046, 3047, 3048, 3049, 3050, 3051, 3052, 3053, 3054,
       3055, 3056, 3057, 3058, 3059, 3060, 3061, 3062, 3063, 3064, 3065,
       3066, 3067, 3068, 3069, 3070])

In [46]:
# indices where new groups of overlapping polygons start
inds1 = np.sort(np.hstack((inds[np.where(np.diff(inds)>10)[0]],inds[np.where(np.diff(inds)>10)[0]+1])))
if len(inds)>0:
    inds1 = np.hstack((inds[0],inds1,inds[-1]))
inds1

array([ 530,  544,  582,  593, 1644, 1664, 3030, 3070])

### 5.5. Fix overlapping polygons

In [48]:
# here we do the actual 'repolygonization'
# 'cropped_polys_new' is the list of polygons in which the overlapping polygons have been replaced
cropped_polys_new = cropped_polys[:] # note that lists are mutable, so we need to make a copy
new_poly_inds = []
pad = 10
crit_dist = 100

for i in range(int(len(inds1)/2)):
    i1 = inds1[2*i]
    i2 = inds1[2*i+1]
    bend, x1, x2, y1, y2 = cm.repolygonize_bend(cropped_polys, cropped_polys_new, i1, i2, pad,
                                                crit_dist, new_poly_inds, x, y)

In [51]:
from descartes import PolygonPatch

# plotting for QC:
fig = plt.figure()
ax = fig.add_subplot(111)
for poly in cropped_polys_new:
    if len(poly.exterior.coords)>0:
        ax.add_patch(PolygonPatch(poly,facecolor='none',edgecolor='k'))
plt.plot(lbx,lby,'k.')
plt.plot(rbx,rby,'k.')
plt.plot(x, y,'b.-')
plt.axis('equal');

In [56]:
plt.figure()
plt.plot(cropped_polys_new[0].exterior.xy[0], cropped_polys_new[0].exterior.xy[1], '.-')

In [57]:
# usually the first few and the last few polygons are not "good", so here we get rid of them, 
# as well as truncate the centerline. Note that it is a good idea to get rid of the first and last polygons,
# even if they look fine at first sight
cropped_polys_new = cropped_polys_new[1:-4]
x = x[1:-4]
y = y[1:-4]

### 5.6. Simplify polygons to 4 corner points

In [58]:
cropped_polys_new = cm.simplify_all_polygons(cropped_polys_new, deltas=25.0)

### 5.7. Create new bank coordinates from polygons

In [59]:
rbxn, rbyn, lbxn, lbyn = cm.create_new_bank_coordinates(cropped_polys_new, x, y)

### 5.8. Write new bank coordinates to shapefile

In [ ]:
import geopandas as gpd

dirname = '/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/'

gs = gpd.GeoSeries(LineString(np.vstack((x,y)).T))
gs.crs = {'init' :'epsg:32620'}
gs.to_file(dirname+'cline_2018.shp')

gs = gpd.GeoSeries(LineString(np.vstack((rbxn,rbyn)).T))
gs.crs = {'init' :'epsg:32620'}
gs.to_file(dirname+'rb_2018.shp')

gs = gpd.GeoSeries(LineString(np.vstack((lbxn,lbyn)).T))
gs.crs = {'init' :'epsg:32620'}
gs.to_file(dirname+'lb_2018.shp')

### 5.9. Write channel segment polygons to shapefile

In [64]:
# compute channel widths and areas of channel segment polygons 
poly_areas = []
widths = []
count = 0
for poly in cropped_polys_new:
    poly_areas.append(poly.area)
    width1 = np.sqrt((lbxn[count]-rbxn[count])**2 + (lbyn[count]-rbyn[count])**2)
    width2 = np.sqrt((lbxn[count+1]-rbxn[count+1])**2 + (lbyn[count+1]-rbyn[count+1])**2)
    width = 0.5*(width1+width2)
    widths.append(width)
    count = count+1

In [65]:
plt.figure()
plt.plot(widths)

In [ ]:
polydata = {'age': '20180622', 'area': poly_areas, 'width': widths}
df = pd.DataFrame(polydata)
gdf = gpd.GeoDataFrame(df, geometry=cropped_polys_new)
gdf.crs = {'init' :'epsg:32620'}
gdf.to_file(dirname+'polys_2018.shp')